In [41]:
import owlready2 as owl
import pandas as pd

filename_onto_individuals = "../../src/ontology/final-archi-individuals.owl"
onto_individuals = owl.get_ontology(filename_onto_individuals).load()

## Airport synonyms

In [42]:
all_airports = list(owl.default_world.sparql(
    f"""
        PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
        SELECT ?name ?iata ?icao ?latitude ?longitude ?altitude ?country
        WHERE {{
            ?Airport pie:AirportName ?name .
            ?Airport pie:AirportIATA ?iata .
            ?Airport pie:AirportICAOCode ?icao .
            ?Airport pie:AirportGPSLatitude ?latitude .
            ?Airport pie:AirportGPSLongitude ?longitude .
            ?Airport pie:AirportAltitude ?altitude .
            ?Airport pie:AirportCountry ?country .
        }}
    """))
fields = ['name', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'country']
dict_all_airports = [dict(zip(fields, airport_tuple)) for airport_tuple in all_airports]
df_airports = pd.DataFrame(dict_all_airports)

In [43]:
df_airports

,name,iata,icao,latitude,longitude,altitude,country
0,A Coruna Airport,LCG,LECO,43.302059,-8.377250,326,Spain
1,Aachen Merzbruck Airport,AAH,EDKA,50.823051,6.186111,623,Germany
2,Aalborg Airport,AAL,EKYT,57.092781,9.849164,10,Denmark
3,Aarhus Airport,AAR,EKAH,56.300011,10.619000,82,Denmark
4,Aarhus Sea Airport,QEA,EKAC,56.151993,10.247725,1,Denmark
...,...,...,...,...,...,...,...
4609,Zunyi Xinzhou Airport,ZYI,ZUZY,27.811167,107.245972,2723,China
4610,Zurich Airport,ZRH,LSZH,47.464722,8.549167,1416,Switzerland
4611,Zwara Airport,WAX,HLZW,32.933334,12.083333,9,Libya
4612,Zweibrucken Airport,ZQW,EDRZ,49.209518,7.400647,1133,Germany


In [44]:
def split_airport(name):
    return name.replace(' Airport', '')

In [45]:
short_names = df_airports['name'].apply(split_airport).tolist()
full_names = df_airports['name']
icao = df_airports['icao']
iata = df_airports['iata']

In [46]:
str_output = ""
for x in list(zip(icao, short_names, full_names, iata)):
    str_output += f"  - {list(x)}\n"

str_output = str_output.replace("'","")

In [47]:
print(str_output)

  - [LECO, A Coruna, A Coruna Airport, LCG]
  - [EDKA, Aachen Merzbruck, Aachen Merzbruck Airport, AAH]
  - [EKYT, Aalborg, Aalborg Airport, AAL]
  - [EKAH, Aarhus, Aarhus Airport, AAR]
  - [EKAC, Aarhus Sea, Aarhus Sea Airport, QEA]
  - [BGAA, Aasiaat, Aasiaat Airport, JEG]
  - [OIAA, Abadan, Abadan Airport, ABD]
  - [UNAA, Abakan International, Abakan International Airport, ABA]
  - [CYXX, Abbotsford International, Abbotsford International Airport, YXX]
  - [EGPD, Aberdeen International, Aberdeen International Airport, ABZ]
  - [KABR, Aberdeen Regional, Aberdeen Regional Airport, ABR]
  - [OEAB, Abha Regional, Abha Regional Airport, AHB]
  - [DIAP, Abidjan Port Bouet, Abidjan Port Bouet Airport, ABJ]
  - [KABI, Abilene Regional, Abilene Regional Airport, ABI]
  - [KVJI, Abingdon Virginia Highlands, Abingdon Virginia Highlands Airport, VJI]
  - [OMAD, Abu Dhabi Al Bateen Executive, Abu Dhabi Al Bateen Executive Airport, AZI]
  - [OMAM, Abu Dhabi Al Dhafra Air Base, Abu Dhabi Al Dhafra

In [48]:
with open("./airports_synonym.txt", "w", encoding="utf-8") as f:
    f.write(str_output)